In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import seaborn as sns
from pandas_datareader import data as pdr
import yfinance as yfin #workaround as pandas_datareader is not working on yahoo today
yfin.pdr_override()
import matplotlib.pyplot as plt

In [2]:
url = "https://www.fundsexplorer.com.br/ranking"
r = requests.get(url)
df_list = pd.read_html(r.text)
df = df_list[0]
df.shape


(244, 26)

In [3]:
# Verificar quantas colunas possuem dados nulos
# df.isnull().sum()

In [4]:
df = df.replace(np.nan,0)
df = df.drop(columns=['DYPatrimonial','VariaçãoPatrimonial','Rentab. Patr.no Período','Rentab. Patr.Acumulada','VacânciaFinanceira','DY Ano','DY (3M)Média','DY (6M)Média'])

In [5]:
dfColunas = []
naoNumColunas = ['Códigodo fundo','Setor']
for coluna in df:
    if coluna in naoNumColunas:
        pass
    else:
        dfColunas.append(coluna)       

In [6]:
for colunas in dfColunas:
    df[colunas] = [str(x).replace("R$ ","").replace("%","").replace(".","").replace(",",".") for x in df[colunas]]
    df[colunas] = [pd.to_numeric(x) for x in df[colunas]]

In [7]:
# Ajustando VPA para as cadas decimais corretas
df['P/VPA'] = df['P/VPA'] / 1000

In [8]:
def filtroDF(quantidadeAtivos=9, liquidezDiaria=1000,PVPA=1.2,VacânciaFísica=10):
# Filtrando para tirar FII's com alto risco
        # Com mais de 5 ativos. Casos menores do que esse aumentam o risco pela baixa pulverização
        dataFrame = df[(df['QuantidadeAtivos'] > quantidadeAtivos) & 
        # Liquidez diária superior a 1000
                (df['Liquidez Diária'] > liquidezDiaria) &
        # P/VPA até Máximo 1,2 (20% a mais do que o patrimonio)
                (df['P/VPA'] < PVPA) &
        # Vacância inferior a 10% do território
                (df['VacânciaFísica'] < VacânciaFísica)]
        return dataFrame
        
df = filtroDF()

In [9]:
testing = []
listingCarlos = ['BTLG11','HGRY11','VILG11','HSML11','HGBS11','XPML11','KNRI11','HGRE11','HGLG11']
for i in df['Códigodo fundo']:
    if i in listingCarlos:
        testing.append(i)
testing        

['BTLG11', 'XPML11', 'HGBS11', 'HGLG11', 'VILG11']

In [10]:
# Ordernar por critérios de maior importância (na minha opinião)
df = df.sort_values(by=['PatrimônioLíq.','Preço Atual','Dividendo',], ascending = False)
df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (12M)Média,Variação Preço,Rentab.Período,Rentab.Acumulada,PatrimônioLíq.,VPA,P/VPA,VacânciaFísica,QuantidadeAtivos
197,XPLG11,Logística,111.49,750270,0.61,0.58,1.63,3.10,5.83,0.49,-7.00,-6.46,-10.13,2.993276e+09,110.45,1.01,6.70,13
75,HGLG11,Logística,169.99,679370,2.20,1.30,2.34,3.66,7.26,0.61,-0.55,0.74,-2.67,2.739488e+09,148.59,1.14,6.90,17
74,HGBS11,Shoppings,199.31,139200,0.50,0.25,0.62,1.57,2.73,0.23,0.43,0.68,-5.48,2.195761e+09,219.58,0.91,8.41,17
156,HGRU11,Híbrido,118.90,562050,0.90,0.77,1.91,3.52,6.73,0.56,-2.62,-1.87,-5.83,2.169839e+09,117.88,1.01,0.00,16
59,XPML11,Shoppings,105.69,422780,0.40,0.38,0.81,1.95,3.56,0.30,1.54,1.93,-5.05,2.001282e+09,109.91,0.96,5.90,13
179,VILG11,Logística,114.50,397210,0.60,0.55,1.46,2.82,5.46,0.45,-6.82,-6.31,-7.31,1.716538e+09,114.46,1.00,0.40,15
180,VISC11,Shoppings,109.00,296400,0.42,0.39,0.94,2.09,3.80,0.32,-1.93,-1.55,-6.46,1.704682e+09,119.45,0.91,8.70,15
185,BRCO11,Logística,107.05,257390,0.54,0.54,1.52,2.97,5.66,0.47,-8.29,-7.80,-9.35,1.602955e+09,108.46,0.99,5.90,10
138,BBPO11,Lajes Corporativas,113.91,212310,1.08,0.94,2.81,5.64,10.40,0.87,-0.25,0.69,5.87,1.598864e+09,100.43,1.13,0.00,64
69,RBVA11,Outros,112.20,154170,1.24,1.10,2.65,5.02,9.42,0.79,1.91,3.03,4.34,1.320790e+09,113.98,0.98,0.60,72


In [21]:
df['Setor'].value_counts()

Logística             7
Outros                4
Shoppings             3
Híbrido               2
Lajes Corporativas    1
Name: Setor, dtype: int64

In [ ]:
df['Preço Atual'].mean()

In [11]:
# Cotação de cada empresa
df_ibov = pdr.get_data_yahoo("GSFI11.SA", start="2020-02-18", end="2021-07-07")
df_ibov = df_ibov[['Open','Close','Volume']]
df_ibov

Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\Users\ViniciusGoncalvesRap\AppData\Local\Programs\Python\Python39\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "C:\Users\ViniciusGoncalvesRap\AppData\Local\Programs\Python\Python39\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ViniciusGoncalvesRap\AppData\Local\Programs\Python\Python39\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\ViniciusGoncalvesRap\AppData\Local\Programs\Python\Python39\lib\site-packages\yfinance\multi.py", line 169, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "C:\Users\ViniciusGoncalvesRap\AppData\Local\Programs\Python\Python39\lib\site-packages\yfinance\multi.py", line 181, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "C:\Users\Vi

KeyboardInterrupt: 

In [ ]:
# plt.figure(figsize=(14,6))
plt.figure(figsize=(18,6))
df_ibov['Volume'].plot()
plt.show()

plt.figure(figsize=(18,6))
df_ibov['Close'].plot()
plt.show()



In [ ]:
plt.figure(figsize=(15,6))
mask = np.triu(np.ones_like(df.corr(), dtype=bool))
sns.heatmap(df.corr(),annot=True, mask=mask)

In [ ]:
# Criar um loop for para salvar todas as cotações em uma tabela
# Plotar adj close relativo